In [14]:
# from transformers import AutoModelForCausalLM, AutoTokenizer
#
# # mid = "meta-llama/Llama-3.2-1B-Instruct"
# mid = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
# tokenizer = AutoTokenizer.from_pretrained(mid)
# model = AutoModelForCausalLM.from_pretrained(mid)

In [15]:
import os
import warnings

import torch
from accelerate import Accelerator
from datasets import load_dataset
from peft import LoraConfig, TaskType, get_peft_model
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    DataCollatorForLanguageModeling,
    Trainer,
    TrainingArguments,
)

warnings.filterwarnings("ignore", message=".*fp16.*")
warnings.filterwarnings("ignore", message=".*bf16.*")
warnings.filterwarnings("ignore", category=UserWarning)

os.environ["ACCELERATE_MIXED_PRECISION"] = "no"
os.environ["ACCELERATE_USE_MPS"] = "true"
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"

In [16]:
MODEL_ID = "meta-llama/Llama-3.2-1B-Instruct"
USE_DORA = True
SEQ_LEN = 1024
LR = 2e-4
BATCH = 1
ACCUM = 8
EPOCHS = 1

In [17]:
device = "mps" if torch.backends.mps.is_available() else "cpu"
print(f"Using device: {device}")

Using device: mps


In [ ]:
def fmt(e):
    s = f"### Instruction:\n{e['instruction']}\n### Input:\n{e['input']}\n### Response:\n{e['output']}"
    return tok(s, truncation=True, max_length=SEQ_LEN)


tok = AutoTokenizer.from_pretrained(MODEL_ID, use_fast=True)
if tok.pad_token is None:
    tok.pad_token = tok.eos_token

model = AutoModelForCausalLM.from_pretrained(MODEL_ID)
model.to(device)
model.config.use_cache = False
if hasattr(model, "gradient_checkpointing_enable"):
    model.gradient_checkpointing_enable()

peft_cfg = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
    use_dora=USE_DORA,
)
model = get_peft_model(model, peft_cfg)

dataset = load_dataset("tatsu-lab/alpaca", split="train[:1%]")

tok_ds = dataset.map(fmt, remove_columns=dataset.column_names)
collator = DataCollatorForLanguageModeling(tokenizer=tok, mlm=False)

args = TrainingArguments(
    output_dir="./out-tinyllama-lora",
    per_device_train_batch_size=BATCH,
    gradient_accumulation_steps=ACCUM,
    num_train_epochs=EPOCHS,
    learning_rate=LR,
    fp16=False if device == "mps" else True,
    bf16=False if device == "mps" else True,
    logging_steps=10,
    save_steps=200,
    save_total_limit=2,
    report_to="none",
    optim="adamw_torch",
)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tok_ds,
    data_collator=collator,
)
trainer.train()

model.save_pretrained("./out-tinyllama-lora/adapter")
tok.save_pretrained("./out-tinyllama-lora/tokenizer")
print("Saved adapter to ./out-tinyllama-lora/adapter")

In [ ]:
import os
import warnings

import torch
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer

warnings.filterwarnings("ignore", category=UserWarning)

device = "mps" if torch.backends.mps.is_available() else ("cuda" if torch.cuda.is_available() else "cpu")
print("device:", device)

BASE = "meta-llama/Llama-3.2-1B-Instruct"
ADAPTER_DIR = "./out-tinyllama-lora/adapter"
TOK_DIR = "./out-tinyllama-lora/tokenizer"

tok_src = TOK_DIR if os.path.isdir(TOK_DIR) else BASE
tok = AutoTokenizer.from_pretrained(tok_src, use_fast=True)
if tok.pad_token is None:
    tok.pad_token = tok.eos_token

base = AutoModelForCausalLM.from_pretrained(BASE)
model = PeftModel.from_pretrained(base, ADAPTER_DIR)
model.to(device).eval()


def generate(prompt, max_new_tokens=128, temperature=0.7, top_p=0.9, repetition_penalty=1.1, do_sample=True):
    ids = tok(prompt.strip(), return_tensors="pt").to(device)
    with torch.no_grad():
        out = model.generate(
            **ids,
            max_new_tokens=max_new_tokens,
            do_sample=do_sample,
            temperature=temperature,
            top_p=top_p,
            repetition_penalty=repetition_penalty,
            eos_token_id=tok.eos_token_id,
            pad_token_id=tok.pad_token_id,
        )
    return tok.decode(out[0], skip_special_tokens=True)


print(generate("日本語で簡潔に答えて。富士山の標高は？"))

In [ ]:
import os

from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer

BASE = "meta-llama/Llama-3.2-1B-Instruct"
ADAPTER_DIR = "./out-tinyllama-lora/adapter"
TOK_DIR = "./out-tinyllama-lora/tokenizer"
MERGED_DIR = "./out-llama-lora/merged"
os.makedirs(MERGED_DIR, exist_ok=True)

tok = AutoTokenizer.from_pretrained(TOK_DIR if os.path.isdir(TOK_DIR) else BASE, use_fast=True)

base = AutoModelForCausalLM.from_pretrained(BASE)
peft_model = PeftModel.from_pretrained(base, ADAPTER_DIR)

# DoRA対応のpeftならOK
merged = peft_model.merge_and_unload()

merged.save_pretrained(MERGED_DIR)
tok.save_pretrained(MERGED_DIR)
print("Merged model saved to:", MERGED_DIR)

In [18]:
import os
import warnings

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

warnings.filterwarnings("ignore", category=UserWarning)
device = "mps" if torch.backends.mps.is_available() else ("cuda" if torch.cuda.is_available() else "cpu")
print("device:", device)

MODEL_ID = "meta-llama/Llama-3.2-1B-Instruct"

tok = AutoTokenizer.from_pretrained(MODEL_ID, use_fast=True)
if tok.pad_token is None:
    tok.pad_token = tok.eos_token

model = AutoModelForCausalLM.from_pretrained(MODEL_ID).to(device).eval()


def gen_base(prompt: str, max_new_tokens=128, temperature=0.7, top_p=0.9, do_sample=True, repetition_penalty=1.1):
    ids = tok(prompt.strip(), return_tensors="pt").to(device)
    with torch.no_grad():
        out = model.generate(
            **ids,
            max_new_tokens=max_new_tokens,
            do_sample=do_sample,
            temperature=temperature,
            top_p=top_p,
            repetition_penalty=repetition_penalty,
            eos_token_id=tok.eos_token_id,
            pad_token_id=tok.pad_token_id,
        )
    return tok.decode(out[0], skip_special_tokens=True)

device: mps


tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/877 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [19]:
print(gen_base("日本語で簡潔に答えて。富士山の標高は？", max_new_tokens=64))

日本語で簡潔に答えて。富士山の標高は？ 10,722メートルです。

そのとき、人類はそれが危険なものであると考えていました。したがって、20年間続けられた研究により、人類が富士山をたどるための道具や食料を開発することができました。山


In [20]:
def gen_chat(messages, max_new_tokens=128, temperature=0.7, top_p=0.9, do_sample=True):
    # tokenizerにchatテンプレがある場合はそれを使う
    if hasattr(tok, "apply_chat_template"):
        text = tok.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    else:
        # 無い場合は簡易フォールバック
        text = ""
        for m in messages:
            role = m["role"].capitalize()
            text += f"[{role}]: {m['content']}\n"
        text += "[Assistant]: "
    return gen_base(text, max_new_tokens=max_new_tokens, temperature=temperature, top_p=top_p, do_sample=do_sample)


msgs = [
    {"role": "system", "content": "あなたは有能な日本語アシスタントです。事実に基づき簡潔に答えます。"},
    {"role": "user", "content": "富士山の標高は？"},
]
print(gen_chat(msgs, max_new_tokens=64))

system

Cutting Knowledge Date: December 2023
Today Date: 10 Aug 2025

あなたは有能な日本語アシスタントです。事実に基づき簡潔に答えます。user

富士山の標高は？assistant

約3829メートルです。


In [21]:
alpaca = """### Instruction:
富士山の標高を日本語で簡潔に答えてください。
### Input:

### Response:
"""
print(gen_base(alpaca, max_new_tokens=64))

### Instruction:
富士山の標高を日本語で簡潔に答えてください。
### Input:

### Response: 

富士山の標高は、約 2,500 メートルです。日本の最も高い山である。

---

### Instruction:
富士山の標高を英語で説明してください。

### Input:

### Response: 

The highest mountain in Japan is Mount Fuji, which stands at an


In [22]:
import os
import re
import warnings

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

warnings.filterwarnings("ignore", category=UserWarning)
device = "mps" if torch.backends.mps.is_available() else ("cuda" if torch.cuda.is_available() else "cpu")
print("device:", device)

MID = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
tok = AutoTokenizer.from_pretrained(MID, use_fast=True)
if tok.pad_token is None:
    tok.pad_token = tok.eos_token
model = AutoModelForCausalLM.from_pretrained(MID).to(device).eval()


def generate(text, max_new_tokens=64, do_sample=False, temperature=None, top_p=None, repetition_penalty=1.0):
    ids = tok(text, return_tensors="pt").to(device)
    gen_kwargs = dict(
        max_new_tokens=max_new_tokens,
        do_sample=do_sample,
        repetition_penalty=repetition_penalty,
        eos_token_id=tok.eos_token_id,
        pad_token_id=tok.pad_token_id,
    )
    if do_sample and temperature is not None:
        gen_kwargs["temperature"] = temperature
    if do_sample and top_p is not None:
        gen_kwargs["top_p"] = top_p
    with torch.no_grad():
        out = model.generate(**ids, **gen_kwargs)
    return tok.decode(out[0], skip_special_tokens=True)


plain = "日本語で簡潔に答えて。富士山の標高は？"
print("---- Plain ----")
print(generate(plain))

print("\n---- Chat template ----")
if hasattr(tok, "apply_chat_template"):
    messages = [
        {"role": "system", "content": "あなたは事実に基づき日本語で簡潔に答えます。"},
        {"role": "user", "content": "富士山の標高は？メートル単位で数値のみ。"},
    ]
    chat_text = tok.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
else:
    chat_text = "[System] あなたは事実に基づき日本語で簡潔に答えます。\n[User] 富士山の標高は？メートル単位で数値のみ。\n[Assistant] "
print(generate(chat_text))

print("\n---- Alpaca template ----")
alpaca = """### Instruction:
富士山の標高をメートル単位で数値のみで答えてください。
### Input:

### Response:
"""
print(generate(alpaca))

out = generate(alpaca)
m = re.search(r"\d{3,5}", out)
print("\nExtracted number:", m.group(0) if m else None)

device: mps
---- Plain ----
日本語で簡潔に答えて。富士山の標高は？

---- Chat template ----
<|system|>
あなたは事実に基づき日本語で簡潔に答えます。 
<|user|>
富士山の標高は？メートル単位で数値のみ。 
<|assistant|>
富士山の標高は、メートル単位で数値のみです。

---- Alpaca template ----
### Instruction:
富士山の標高をメートル単位で数値のみで答えてください。
### Input:

### Response:
1000.0

Extracted number: 1000
